In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import datetime 
from polygon import StocksClient
from env import POLYGON_API_KEY as api_key
import yfinance as yf
from tqdm import tqdm
from alive_progress import alive_bar
import os
#Polygon api
client = StocksClient(api_key, connect_timeout=10)

In [3]:
days = pd.read_csv('matchDays.csv',index_col=False)
c=50000
data = []
days.columns = ['date','symbol']
hour_4 = datetime.time(hour=4)
hour_9 = datetime.time(hour=9)
hour_16 = datetime.time(hour=16)

#os.mkdir("daily_stocks")

In [102]:
days

,date,symbol
0,03/12/2020,AA
1,03/13/2020,AA
2,03/18/2020,AA
3,03/19/2020,AA
4,03/23/2020,AA
...,...,...
95001,04/14/2023,HUBC
95002,04/14/2023,CRBP
95003,04/14/2023,BITF
95004,04/14/2023,ALLR


# Creating the database

In [122]:
data = []
hour_4 = datetime.time(hour=4)
hour_9 = datetime.time(hour=9)
hour_16 = datetime.time(hour=16)

h=0
with alive_bar(len(days)-c,force_tty=True) as bar:
    for _,i in days[c:].iterrows():
        c+=1
        day = datetime.datetime.strptime(i[0],"%m/%d/%Y") + datetime.timedelta(days=1)
        day_4 = datetime.datetime.combine(day,hour_4)
        day_9 = datetime.datetime.combine(day,hour_9)
        day_16 = datetime.datetime.combine(day,hour_16)
        bar()
        aux = client.get_aggregate_bars(i[1],day_4,day_16,timespan='min')
        if(not os.path.exists(os.path.join("daily_stocks",i[1]))):
            os.mkdir(os.path.join("daily_stocks",i[1]))
        if(aux['status']=='OK' and aux['queryCount']!=0):
            df = pd.DataFrame(aux['results'])
            df['symbol'] = i[1]
            df.to_csv(os.path.join(os.getcwd(),"daily_stocks",i[1],i[0].replace('/','-')+".csv"))

|⚠︎                                       | (!) 1/42263 [0%] in 0.0s (1485.71/s) 


AttributeError: 'float' object has no attribute 'upper'

# Important Functions to get data

In [17]:
#function to get files in a folder
def process_files_in_folder(folder_path):
    filenames =[] 
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            # Process the file here
            filenames.append(file_path)
        elif os.path.isdir(file_path):
            # Recursively call this function for subfolders
            process_files_in_folder(file_path)
    return filenames


#Function to get data from an specific date and stock
def getDataDayStock(stock,date):
    day = datetime.datetime.strptime(date,"%m-%d-%Y") + datetime.timedelta(days=1)
    day_4 = datetime.datetime.combine(day,hour_4)
    day_9 = datetime.datetime.combine(day,hour_9)
    day_16 = datetime.datetime.combine(day,hour_16)
    aux = client.get_aggregate_bars(stock,day_4,day_16,timespan='min')
    if(not os.path.exists(os.path.join("daily_stocks",stock))):
        os.mkdir(os.path.join("daily_stocks",stock))
    if(aux['status']=='OK' and aux['queryCount']!=0):
        df = pd.DataFrame(aux['results'])
        df['symbol'] = stock
        df.to_csv(os.path.join(os.getcwd(),"daily_stocks",stock,date+".csv"))
    return pd.read_csv(os.path.join(os.getcwd(),"daily_stocks",stock,date+".csv"),index_col=False)

#function to manage changes because of a split
def checkChanges(path):
    fileName = process_files_in_folder(path)
    aux = fileName[0].split("/")[1:]
    stock = aux[0]
    date = aux[1].split(".")[0]
    temp_file = pd.read_csv(os.path.join(os.getcwd(),"daily_stocks",stock,date+".csv"),index_col=False)
    df = getDataDayStock(stock,date)
    if (temp_file.iloc[0].o == df.iloc[0].o):
        print("No changes in "+stock)
    else:
        for i in fileName:
            aux = i.split("/")[1:]
            stock = aux[0]
            date = aux[1].split(".")[0]
            getDataDayStock(stock,date)

def checkChangesDividends(path):
    fileName = process_files_in_folder(path)
    aux = fileName[0].split("/")[1:]
    stock = aux[0]
    date = aux[1].split(".")[0]
    temp_file = pd.read_csv(os.path.join(os.getcwd(),"daily_stocks",stock,date+".csv"),index_col=False)
    df = getDataDayStock(stock,date)


In [18]:
checkChanges("daily_stocks/AA")

No changes in AA


In [123]:
# import necessary libraries
import pandas as pd
import os
import glob
import re
path = os.path.join(os.getcwd(),"daily_stocks")
stocks_data = pd.DataFrame()
data_to_analyze = pd.DataFrame()

    
folders = next(os.walk("daily_stocks/"))[1]
data = pd.DataFrame()
with alive_bar(len(folders),force_tty=True) as bar:
    for i in folders: 
        bar()
        pattern = "^[A-D].*"
        match = re.match(pattern, i)
        if match:
            files = process_files_in_folder('daily_stocks/'+i)
            for i in files:
                data = pd.concat([data,pd.read_csv(i)])

|⚠︎                                       | (!) 7/5579 [0%] in 0.0s (10647.79/s) 


NameError: name 'process_files_in_folder' is not defined

In [82]:
data

,Unnamed: 0,v,vw,o,c,h,l,t,n,symbol
0,0,12324.0,36.6607,36.6600,36.6600,36.660,36.6600,1645108200000,70,AMCX
1,1,885.0,36.6457,36.5895,36.6500,36.745,36.5895,1645108260000,18,AMCX
2,2,3365.0,36.8999,36.8200,36.7803,36.980,36.7000,1645108320000,15,AMCX
3,3,2328.0,36.7223,36.6800,36.7200,36.850,36.6800,1645108380000,42,AMCX
4,4,1603.0,36.8695,36.8155,36.8750,36.945,36.8155,1645108440000,19,AMCX
...,...,...,...,...,...,...,...,...,...,...
155,155,144.0,16.9219,16.9300,16.9300,16.930,16.9300,1623254160000,4,ACRS
156,156,208.0,16.9105,16.9300,16.8900,16.930,16.8900,1623254220000,5,ACRS
157,157,817.0,16.9334,16.9400,16.9200,16.940,16.9200,1623254280000,16,ACRS
158,158,621.0,16.9620,16.9500,16.9750,16.975,16.9500,1623254340000,17,ACRS


In [93]:
#start premarket hour
hour_4 = datetime.time(hour=4)
#start market hour
hour_9 = datetime.time(hour=9)
#closed time 
hour_16 = datetime.time(hour=16)
data_market = []

data.columns = ["index","volume","w_volumen","open","close","high","low","time","#transactions","symbol"]
datetime.date.fromtimestamp(data.time.iloc[0]/1000)
#data['time'] = pd.to_datetime(data.time)
#print(data.loc[0].head())

datetime.date(2022, 2, 17)

In [83]:
#get red closes
red_closes = []
last_symbol = ""
last_date = None
data = []
import math
with alive_bar(len(days)-(30499+22242),force_tty=True) as bar:
    for i in range(30499+22242,len(days)):
        bar()
        date_aux = datetime.datetime.strptime(days.iloc[i]['date'],'%m/%d/%Y')
        if(isinstance((days.iloc[i]['symbol']),str) or not math.isnan(days.iloc[i]['symbol'])):
            aux = client.get_daily_open_close(days.iloc[i]['symbol'],date_aux)
            data.append(aux)
            if (aux['close']-aux['open'])/aux['open'] <0:
                red_closes.append(days.iloc[i])

|████████████████████████████████████████| 42265/42265 [100%] in 2:13:49.9 (5.26


In [16]:
print(days.)

             date symbol
0      03/12/2020     AA
1      03/13/2020     AA
2      03/18/2020     AA
3      03/19/2020     AA
4      03/23/2020     AA
...           ...    ...
95001  04/14/2023   HUBC
95002  04/14/2023   CRBP
95003  04/14/2023   BITF
95004  04/14/2023   ALLR
95005  04/14/2023   MNMD

[95006 rows x 2 columns]


In [22]:
red_closes1=red_closes
x = pd.read_csv("data3.csv")
print(len(x))
print(len(days))

22241
95006


In [62]:
pd.DataFrame(data).to_csv("data3.csv")
pd.DataFrame(np.array(red_closes)).to_csv("red_3.csv")

In [71]:
red_closes = pd.DataFrame(np.array(red_closes))
red_closes.columns=['date','symbol']
red_closes.to_csv('red_3.csv')
print(type(red_closes))

<class 'pandas.core.frame.DataFrame'>


In [73]:
red1 = pd.read_csv('red_1.csv')
red2 = pd.read_csv('red_2.csv')
red3 = pd.read_csv('red_3.csv')

,Unnamed: 0,date,symbol
0,0,03/18/2020,NAC
1,1,03/18/2020,NAD
2,2,02/10/2021,NAII
3,3,08/16/2021,NAII
4,4,02/25/2020,NAIL
5,5,02/28/2020,NAIL
6,6,03/09/2020,NAIL
7,7,03/11/2020,NAIL
8,8,03/12/2020,NAIL
9,9,03/13/2020,NAIL


In [101]:
x = pd.concat([red1,red2,red3])
len(x['symbol'])

25789

In [124]:
with alive_bar(len(x),force_tty=True) as bar:
    for i in x.iterrows():
        bar()
        if(isinstance((i[1]['symbol']),str) or not math.isnan(i[1]['symbol'])):
            if(os.path.exists('daily_stocks/' + i[1]['symbol'] +"/"+i[1]['date'].replace("/","-") +".csv")):
                aux = pd.read_csv('daily_stocks/' + i[1]['symbol'] +"/"+i[1]['date'].replace("/","-") +".csv")
            else:
                date = i[1]['date']
                day = datetime.datetime.strptime(date,"%m/%d/%Y") + datetime.timedelta(days=1)
                day_4 = datetime.datetime.combine(day,hour_4)
                day_9 = datetime.datetime.combine(day,hour_9)
                day_16 = datetime.datetime.combine(day,hour_16)
                aux = client.get_aggregate_bars(i[1]['symbol'],day_4,day_16,timespan='min')
                if(aux['status']=='OK' and aux['queryCount']!=0):
                    df = pd.DataFrame(aux['results'])
                    df['symbol'] = i[1]
                    df.to_csv(os.path.join(os.getcwd(),"daily_stocks",i[1],i[0].replace('/','-')+".csv"))

|████████████████████████████████████████| 25789/25789 [100%] in 10:44.8 (39.39/
